In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY 가 .env에 없어요. (.env에 OPENAI_API_KEY=... 추가)"

In [6]:
from typing import List, Dict, Literal, TypedDict
from pydantic import BaseModel, Field
from typing_extensions import Annotated

Valence = Annotated[float, Field(ge=-1.0, le=1.0)]      # 정서의 밝기(부정 - 긍정)
Arousal = Annotated[float, Field(ge=0.0, le=1.0)]       # 각성도(에너지)
Confidence = Annotated[float, Field(ge=0.0, le=1.0)]    # 확신도
BPM     = Annotated[int,   Field(ge=50,  le=140)]       # 분당 비트수(BPM)
DurSec  = Annotated[int,   Field(ge=5,   le=60)]        # 음악 길이(5초 - 60초) ※임시

class EmotionResult(BaseModel):     # 감정 분석 결과의 틀
    primary: str                    # 행복/슬픔/불안 같은 주요 감정 라벨 (문자열)
    valence: Valence = 0.0          # 밝기(-1~1) 기본값 0
    arousal: Arousal = 0.5          # 각성도(0~1) 기본값 0.5
    confidence: Confidence = 0.7    # 확신도(0~1) 기본값 0.7
    reasons: str = "—"              # 감정 판단의 근거 요약 기본은 —

class MusicBrief(BaseModel):        # 음악 생성 전에 필요한 설계서(브리프) 스키마
    mood: str                       # calm/uplifting/melancholic 같은 전반 무드
    bpm: BPM = 90                   # 50~140 범위의 정수 기본 90
    key: str = "C major"            # 조성. 예: “C major”, “A minor”
    duration_sec: DurSec = 20       # 5~60 범위의 정수(초). 기본 20
    instruments: List[str] = []     # 핵심 악기들. 예: ["piano", "strings"].
    style_tags: List[str] = []      # 스타일 태그. 예: ["ambient", "cinematic"].
    prompt: str                     # 텍스트-투-뮤직 모델에 줄 영문 프롬프트 (필수)

class GraphState(TypedDict, total=False):
    user_text: str                              # 사용자 입력 문장
    use_external: bool                          # 외부 모델(예: Replicate) 쓸지 여부
    emotion: EmotionResult                      # 위 스키마로 검증된 감정 결과
    brief: MusicBrief                           # 위 스키마로 검증된 음악 브리프
    audio_path: str                             # 생성된 오디오 파일 경로
    provider_used: Literal["replicate", "mock"] # 어떤 방식으로 생성했나
    meta: Dict                                  # 그 외 로그/디버그용 메타데이터


In [ ]:
def get_llm():
    # OpenAI Chat 모델을 래핑한 LangChain 객체
    # temperature=0.2 → 더 일관된(덜 랜덤) 출력
    return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

#스키마 강제(Structured Output), 감정 라벨링: 0.0 ~ 0.3
#음악 브리프 같이 살짝 창의성 필요: 0.3 ~ 0.6
#가사/스토리처럼 창작성 극대화: 0.7 ~ 1.0 (형식 깨질 수 있음)


In [ ]:
def analyze_emotion_node(state: GraphState) -> GraphState: # 감정 분석 노드
    llm = get_llm()

    # 시스템 지시문: 역할/출력 형식을 LLM에게 명확히 지정
    sys = (
        "당신은 심리 정서를 요약하는 분석가입니다. "
        "사용자 텍스트로부터 주요 감정을 한 단어(또는 짧은 구)로 도출하고, "
        "valence(-1~1)와 arousal(0~1), confidence(0~1)를 추정하세요. "
        "반드시 주어진 JSON 스키마(EmotionResult)에 맞춰 응답하세요."
    )

    # 사용자 입력은 state에서 꺼냄 (그래프의 '가방'에서 user_text를 읽어오는 느낌)
    user_msg = state['user_text']

    # LLM 출력이 EmotionResult 스키마에 '자동으로 검증/파싱'되도록 래핑
    structured = llm.with_structured_output(EmotionResult)

    # 메시지 형식: system + user
    result = structured.invoke([
        {"role": "system", "content": sys},
        {"role": "user",   "content": user_msg}
    ])

    # 결과를 state에 저장하고 다음 노드가 쓰게 한다
    state['emotion'] = result
    return state


In [11]:
def compose_brief_node(state: GraphState) -> GraphState:
    llm = get_llm()                            # 1) 같은 LLM 설정 재사용
    emo: EmotionResult = state["emotion"]      # 2) 방금 저장한 감정 결과 꺼내기

    # 3) 시스템 지시문: 출력 형식(브리프 규칙)과 제약(bpm, duration, key)을 설명
    sys = (
        "너는 음악 감독이다. 아래 감정 분석 결과와 사용자 텍스트를 참고해, "
        "치유/안정 목적의 짧은 BGM에 적합한 Music Brief를 JSON으로 만들어라. "
        "bpm은 50~140, duration_sec는 5~60 사이여야 하며, key는 'C major' 같은 형식. "
        "prompt는 영어로, 핵심 악기/무드/다이내믹을 간결히 포함할 것."
    )

    # 4) 사용자 메시지: 감정 수치와 원문 텍스트를 함께 전달
    usr = (
        f"# Emotion\n"
        f"primary={emo.primary}, valence={emo.valence}, arousal={emo.arousal}, confidence={emo.confidence}\n\n"
        f"# Text\n{state['user_text']}\n"
    )

    # 5) MusicBrief 스키마로 구조화 출력 강제
    structured = llm.with_structured_output(MusicBrief)
    brief = structured.invoke([
        {"role": "system", "content": sys},
        {"role": "user",   "content": usr}
    ])

    state["brief"] = brief                    # 6) 다음 노드에서 쓰도록 가방에 담기
    return state


In [12]:
# --- Step 7: 음악 생성 노드 (Replicate or Mock) ---
import time
import wave
import struct
import requests
import numpy as np
from typing import Optional, Tuple

# 7-1) 간단 WAV 저장기 (16-bit PCM, 모노)
def write_wav(path: str, sr: int, audio: np.ndarray):
    data = np.clip(audio, -1.0, 1.0)
    data = (data * 32767.0).astype(np.int16)  # float[-1,1] -> int16
    with wave.open(path, 'wb') as wf:
        wf.setnchannels(1)      # mono
        wf.setsampwidth(2)      # 16-bit
        wf.setframerate(sr)
        wf.writeframes(data.tobytes())

# 7-2) Mock 합성기: 간단한 사인파 멜로디(스케일 + BPM 기반)
NOTE_FREQ = {
    "C4": 261.63, "D4": 293.66, "E4": 329.63, "F4": 349.23,
    "G4": 392.00, "A4": 440.00, "B4": 493.88,
    "C5": 523.25, "D5": 587.33, "E5": 659.25
}
SCALES = {
    "C_major": ["C4","D4","E4","F4","G4","A4","B4","C5","D5","E5"],
    "A_minor": ["A4","B4","C5","D5","E5","G4","A4","C5","D5"],
}
def _sine(freq, t, sr):  # 기본 사인파
    return np.sin(2*np.pi*freq*t)

def synth_mock(brief: MusicBrief, seed: int = 42) -> Tuple[int, np.ndarray]:
    sr = 44100
    dur = int(brief.duration_sec)
    t = np.linspace(0, dur, int(sr*dur), endpoint=False)

    # 키에 따라 간단 스케일 선택
    scale_name = "C_major"
    if "minor" in brief.key.lower():
        scale_name = "A_minor"
    scale = [NOTE_FREQ.get(n, 440.0) for n in SCALES.get(scale_name, SCALES["C_major"])]

    bpm = int(brief.bpm)
    beat = 60.0 / max(1, bpm)
    note_len = beat * 0.95

    audio = np.zeros_like(t)
    rng = np.random.default_rng(seed)
    pos = 0.0

    while pos < dur:
        f = float(rng.choice(scale))
        on = pos
        off = min(pos + note_len, dur)
        seg = (t >= on) & (t < off)
        tt = t[seg] - on

        # 사인 + 약간의 하모닉
        sig = 0.6*_sine(f, tt, sr) + 0.25*_sine(2*f, tt, sr) + 0.15*_sine(3*f, tt, sr)

        # 간단 attack/release
        attack = int(0.01*sr)
        release = int(0.08*sr)
        env = np.ones_like(sig)
        env[:attack] = np.linspace(0, 1, attack, endpoint=False)
        if release < len(env):
            env[-release:] = np.linspace(1, 0, release, endpoint=False)

        audio[seg] += sig * env
        pos += beat

    # 소프트 클리핑 + 노멀라이즈
    audio = np.tanh(audio)
    audio /= (np.max(np.abs(audio)) + 1e-8)
    return sr, audio.astype(np.float32)

# 7-3) (옵션) Replicate MusicGen 호출
def generate_with_replicate(prompt: str, duration: int) -> Optional[str]:
    token = os.getenv("REPLICATE_API_TOKEN")
    if not token:
        return None
    try:
        import replicate  # 필요: pip install replicate
    except Exception:
        return None

    client = replicate.Client(api_token=token)
    # 주의: 실제 환경에선 최신 식별자/버전 명시 권장
    output = client.run(
        "meta/musicgen",
        input={"prompt": prompt, "duration": duration}
    )
    if isinstance(output, list) and output:
        url = output[0]
        # URL의 바이너리를 파일로 저장
        os.makedirs("outputs", exist_ok=True)
        out = f"outputs/musicgen_{int(time.time())}.wav"
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(out, "wb") as f:
            f.write(r.content)
        return out
    return None

# 7-4) LangGraph 노드: brief -> 오디오 파일 생성
def generate_music_node(state: GraphState) -> GraphState:
    brief: MusicBrief = state["brief"]
    use_ext = state.get("use_external", False)

    os.makedirs("outputs", exist_ok=True)
    provider = "mock"
    audio_path = None

    # (1) 외부 모델 시도
    if use_ext:
        audio_path = generate_with_replicate(brief.prompt, int(brief.duration_sec))
        if audio_path:
            provider = "replicate"

    # (2) 실패/미사용이면 Mock 합성
    if audio_path is None:
        sr, audio = synth_mock(brief)
        audio_path = f"outputs/mock_{int(time.time())}.wav"
        write_wav(audio_path, sr, audio)
        provider = "mock"

    # (3) 결과를 state에 적재 (다음 단계나 최종 출력에 사용)
    state["audio_path"] = audio_path
    state["provider_used"] = provider
    state["meta"] = {
        "emotion": state["emotion"].model_dump(),
        "brief": state["brief"].model_dump(),
        "provider": provider,
        "path": audio_path,
    }
    return state


In [20]:
def build_graph_with_start():
    builder = StateGraph(GraphState)

    builder.add_node("analyze_emotion", analyze_emotion_node)
    builder.add_node("compose_brief",  compose_brief_node)
    builder.add_node("generate_music", generate_music_node)

    builder.add_edge(START, "analyze_emotion")
    builder.add_edge("analyze_emotion", "compose_brief")
    builder.add_edge("compose_brief",  "generate_music")
    builder.add_edge("generate_music", END)

    app = builder.compile()
    viz = app.get_graph()          # 🔁 여기! builder가 아니라 app에서 get_graph()
    return app, viz


In [21]:
app, viz = build_graph_with_start()

state = {
    "user_text": "오늘 면접을 앞두고 떨려요. 마음을 차분하게 다잡고 싶어요.",
    "use_external": False,   # True면 Replicate 사용(토큰 필요)
}

final = app.invoke(state)

print("=== Emotion ===")
print(final["emotion"].model_dump())

print("\n=== Music Brief ===")
print(final["brief"].model_dump())

print("\n=== Provider Used ===")
print(final["provider_used"])

print("\n=== Audio Path ===")
print(final["audio_path"])


=== Emotion ===
{'primary': '긴장', 'valence': -0.5, 'arousal': 0.7, 'confidence': 0.8, 'reasons': '면접을 앞두고 느끼는 긴장감과 불안감 때문입니다.'}

=== Music Brief ===
{'mood': 'calming', 'bpm': 70, 'key': 'C major', 'duration_sec': 30, 'instruments': ['piano', 'strings'], 'style_tags': [], 'prompt': 'A soothing background music to help ease tension before an interview, featuring gentle piano and soft strings.'}

=== Provider Used ===
mock

=== Audio Path ===
outputs/mock_1757666887.wav
